<a href="https://colab.research.google.com/github/matheus-teixeira-viana/hackathon_dm/blob/dm/dataprep_dm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação Pyspark
!pip install pyspark

In [ ]:
# Mapear nosso Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Criação da sessão com o SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("App_Hackaton_dm") \
    .config('spark.ui.port', '4050') \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [ ]:
# Leitura dos dados
df_dm = spark.read.parquet("/content/gdrive/My Drive/hackathon_24/fato_total_risco_dm")

df_dm.createOrReplaceTempView("df_dm")

In [ ]:
df_dm.printSchema()

root
 |-- ch_cpf_hash: string (nullable = true)
 |-- dt_min_adesao: date (nullable = true)
 |-- dt_max_adesao: date (nullable = true)
 |-- vl_qnt_cartao: integer (nullable = true)
 |-- vl_min_score_colletion: integer (nullable = true)
 |-- vl_max_score_colletion: integer (nullable = true)
 |-- vl_min_dias_atraso: integer (nullable = true)
 |-- vl_max_dias_atraso: integer (nullable = true)
 |-- vl_min_saldo_devedor: decimal(26,6) (nullable = true)
 |-- vl_max_saldo_devedor: decimal(26,6) (nullable = true)
 |-- vl_total_saldo_devedor: decimal(26,6) (nullable = true)
 |-- vl_min_limite: decimal(15,2) (nullable = true)
 |-- vl_max_limite: decimal(15,2) (nullable = true)
 |-- vl_total_limite: decimal(15,2) (nullable = true)
 |-- ds_behaviour: string (nullable = true)
 |-- fl_cpf_ativo: string (nullable = true)
 |-- dt_ultima_compra: date (nullable = true)
 |-- fl_tem_dm_app: string (nullable = true)
 |-- dt_cadastro_dmapp: date (nullable = true)
 |-- dt_data_ultimo_acesso_app: date (nullabl

In [ ]:
# Segmentacao para fins de teste E2E do script

#df_dm = spark.sql("""select * from df_dm where dt_fechamento = '2024-09-30' """)
#df_dm.createOrReplaceTempView("df_dm")

In [ ]:
#Dataprep
# 1. Prepara base para criação de variáveis numéricas
df_dm_dataprep_num_var = spark.sql(""" SELECT ch_cpf_hash
    ,CASE WHEN vl_qnt_cartao IS NULL THEN -99 ELSE vl_qnt_cartao END AS vl_qnt_cartao
    ,CASE WHEN vl_min_score_colletion IS NULL THEN -99 ELSE vl_min_score_colletion END AS vl_min_score_colletion
    ,CASE WHEN vl_max_score_colletion IS NULL THEN -99 ELSE vl_max_score_colletion END AS vl_max_score_colletion
    ,CASE WHEN vl_min_dias_atraso IS NULL THEN -99 ELSE vl_min_dias_atraso END AS vl_min_dias_atraso
    ,CASE WHEN vl_max_dias_atraso IS NULL THEN -99 ELSE vl_max_dias_atraso END AS vl_max_dias_atraso
    ,CASE WHEN vl_min_saldo_devedor IS NULL THEN -99 ELSE vl_min_saldo_devedor END AS vl_min_saldo_devedor
    ,CASE WHEN vl_max_saldo_devedor IS NULL THEN -99 ELSE vl_max_saldo_devedor END AS vl_max_saldo_devedor
    ,CASE WHEN vl_total_saldo_devedor IS NULL THEN -99 ELSE vl_total_saldo_devedor END AS vl_total_saldo_devedor
    ,CASE WHEN vl_min_limite IS NULL THEN -99 ELSE vl_min_limite END AS vl_min_limite
    ,CASE WHEN vl_max_limite IS NULL THEN -99 ELSE vl_max_limite END AS vl_max_limite
    ,CASE WHEN vl_total_limite IS NULL THEN -99 ELSE vl_total_limite END AS vl_total_limite
    ,CASE WHEN ds_behaviour IS NULL THEN -99 ELSE ds_behaviour END AS ds_behaviour
    ,CASE WHEN vl_valor_ultimo_movimento IS NULL THEN -99 ELSE vl_valor_ultimo_movimento END AS vl_valor_ultimo_movimento
    ,CASE WHEN vl_saldo_dm_conta IS NULL THEN -99 ELSE vl_saldo_dm_conta END AS vl_saldo_dm_conta
    ,CASE WHEN id_cliente_nivel IS NULL THEN -99 ELSE id_cliente_nivel END AS id_cliente_nivel
    ,CASE WHEN id_ciclo IS NULL THEN -99 ELSE id_ciclo END AS id_ciclo
    ,CASE WHEN vl_dias_atraso_dmcred IS NULL THEN -99 ELSE vl_dias_atraso_dmcred END AS vl_dias_atraso_dmcred
    ,CASE WHEN vl_saldo_devedor_dmcred IS NULL THEN -99 ELSE vl_saldo_devedor_dmcred END AS vl_saldo_devedor_dmcred
    ,CASE WHEN vl_qnt_compras_dmpag IS NULL THEN -99 ELSE vl_qnt_compras_dmpag END AS vl_qnt_compras_dmpag
    ,CASE WHEN vl_total_compra IS NULL THEN -99 ELSE vl_total_compra END AS vl_total_compra
    ,CASE WHEN vl_dias_atraso_dmpag IS NULL THEN -99 ELSE vl_dias_atraso_dmpag END AS vl_dias_atraso_dmpag
    ,CASE WHEN vl_saldo_devedor_dmpag IS NULL THEN -99 ELSE vl_saldo_devedor_dmpag END AS vl_saldo_devedor_dmpag
    ,CASE WHEN vl_Farmacia_qnt_cartao IS NULL THEN -99 ELSE vl_Farmacia_qnt_cartao END AS vl_Farmacia_qnt_cartao
    ,CASE WHEN vl_Farmacia_min_limite IS NULL THEN -99 ELSE vl_Farmacia_min_limite END AS vl_Farmacia_min_limite
    ,CASE WHEN vl_Farmacia_max_limite IS NULL THEN -99 ELSE vl_Farmacia_max_limite END AS vl_Farmacia_max_limite
    ,CASE WHEN vl_Farmacia_total_limite IS NULL THEN -99 ELSE vl_Farmacia_total_limite END AS vl_Farmacia_total_limite
    ,CASE WHEN vl_Farmacia_min_score_colletion IS NULL THEN -99 ELSE vl_Farmacia_min_score_colletion END AS vl_Farmacia_min_score_colletion
    ,CASE WHEN vl_Farmacia_max_score_colletion IS NULL THEN -99 ELSE vl_Farmacia_max_score_colletion END AS vl_Farmacia_max_score_colletion
    ,CASE WHEN vl_Farmacia_min_dias_atraso IS NULL THEN -99 ELSE vl_Farmacia_min_dias_atraso END AS vl_Farmacia_min_dias_atraso
    ,CASE WHEN vl_Farmacia_max_dias_atraso IS NULL THEN -99 ELSE vl_Farmacia_max_dias_atraso END AS vl_Farmacia_max_dias_atraso
    ,CASE WHEN vl_Farmacia_min_saldo_devedor IS NULL THEN -99 ELSE vl_Farmacia_min_saldo_devedor END AS vl_Farmacia_min_saldo_devedor
    ,CASE WHEN vl_Farmacia_max_saldo_devedor IS NULL THEN -99 ELSE vl_Farmacia_max_saldo_devedor END AS vl_Farmacia_max_saldo_devedor
    ,CASE WHEN vl_Farmacia_total_saldo_devedor IS NULL THEN -99 ELSE vl_Farmacia_total_saldo_devedor END AS vl_Farmacia_total_saldo_devedor
    ,CASE WHEN vl_Mercado_qnt_cartao IS NULL THEN -99 ELSE vl_Mercado_qnt_cartao END AS vl_Mercado_qnt_cartao
    ,CASE WHEN vl_Mercado_min_limite IS NULL THEN -99 ELSE vl_Mercado_min_limite END AS vl_Mercado_min_limite
    ,CASE WHEN vl_Mercado_max_limite IS NULL THEN -99 ELSE vl_Mercado_max_limite END AS vl_Mercado_max_limite
    ,CASE WHEN vl_Mercado_total_limite IS NULL THEN -99 ELSE vl_Mercado_total_limite END AS vl_Mercado_total_limite
    ,CASE WHEN vl_Mercado_min_score_colletion IS NULL THEN -99 ELSE vl_Mercado_min_score_colletion END AS vl_Mercado_min_score_colletion
    ,CASE WHEN vl_Mercado_max_score_colletion IS NULL THEN -99 ELSE vl_Mercado_max_score_colletion END AS vl_Mercado_max_score_colletion
    ,CASE WHEN vl_Mercado_min_dias_atraso IS NULL THEN -99 ELSE vl_Mercado_min_dias_atraso END AS vl_Mercado_min_dias_atraso
    ,CASE WHEN vl_Mercado_max_dias_atraso IS NULL THEN -99 ELSE vl_Mercado_max_dias_atraso END AS vl_Mercado_max_dias_atraso
    ,CASE WHEN vl_Mercado_min_saldo_devedor IS NULL THEN -99 ELSE vl_Mercado_min_saldo_devedor END AS vl_Mercado_min_saldo_devedor
    ,CASE WHEN vl_Mercado_max_saldo_devedor IS NULL THEN -99 ELSE vl_Mercado_max_saldo_devedor END AS vl_Mercado_max_saldo_devedor
    ,CASE WHEN vl_Mercado_total_saldo_devedor IS NULL THEN -99 ELSE vl_Mercado_total_saldo_devedor END AS vl_Mercado_total_saldo_devedor
    ,CASE WHEN vl_Construcao_qnt_cartao IS NULL THEN -99 ELSE vl_Construcao_qnt_cartao END AS vl_Construcao_qnt_cartao
    ,CASE WHEN vl_Construcao_min_limite IS NULL THEN -99 ELSE vl_Construcao_min_limite END AS vl_Construcao_min_limite
    ,CASE WHEN vl_Construcao_max_limite IS NULL THEN -99 ELSE vl_Construcao_max_limite END AS vl_Construcao_max_limite
    ,CASE WHEN vl_Construcao_total_limite IS NULL THEN -99 ELSE vl_Construcao_total_limite END AS vl_Construcao_total_limite
    ,CASE WHEN vl_Construcao_min_score_colletion IS NULL THEN -99 ELSE vl_Construcao_min_score_colletion END AS vl_Construcao_min_score_colletion
    ,CASE WHEN vl_Construcao_max_score_colletion IS NULL THEN -99 ELSE vl_Construcao_max_score_colletion END AS vl_Construcao_max_score_colletion
    ,CASE WHEN vl_Construcao_min_dias_atraso IS NULL THEN -99 ELSE vl_Construcao_min_dias_atraso END AS vl_Construcao_min_dias_atraso
    ,CASE WHEN vl_Construcao_max_dias_atraso IS NULL THEN -99 ELSE vl_Construcao_max_dias_atraso END AS vl_Construcao_max_dias_atraso
    ,CASE WHEN vl_Construcao_min_saldo_devedor IS NULL THEN -99 ELSE vl_Construcao_min_saldo_devedor END AS vl_Construcao_min_saldo_devedor
    ,CASE WHEN vl_Construcao_max_saldo_devedor IS NULL THEN -99 ELSE vl_Construcao_max_saldo_devedor END AS vl_Construcao_max_saldo_devedor
    ,CASE WHEN vl_Construcao_total_saldo_devedor IS NULL THEN -99 ELSE vl_Construcao_total_saldo_devedor END AS vl_Construcao_total_saldo_devedor
    ,CASE WHEN vl_Bandeirado_qnt_cartao IS NULL THEN -99 ELSE vl_Bandeirado_qnt_cartao END AS vl_Bandeirado_qnt_cartao
    ,CASE WHEN vl_Bandeirado_min_limite IS NULL THEN -99 ELSE vl_Bandeirado_min_limite END AS vl_Bandeirado_min_limite
    ,CASE WHEN vl_Bandeirado_max_limite IS NULL THEN -99 ELSE vl_Bandeirado_max_limite END AS vl_Bandeirado_max_limite
    ,CASE WHEN vl_Bandeirado_total_limite IS NULL THEN -99 ELSE vl_Bandeirado_total_limite END AS vl_Bandeirado_total_limite
    ,CASE WHEN vl_Bandeirado_min_score_colletion IS NULL THEN -99 ELSE vl_Bandeirado_min_score_colletion END AS vl_Bandeirado_min_score_colletion
    ,CASE WHEN vl_Bandeirado_max_score_colletion IS NULL THEN -99 ELSE vl_Bandeirado_max_score_colletion END AS vl_Bandeirado_max_score_colletion
    ,CASE WHEN vl_Bandeirado_min_dias_atraso IS NULL THEN -99 ELSE vl_Bandeirado_min_dias_atraso END AS vl_Bandeirado_min_dias_atraso
    ,CASE WHEN vl_Bandeirado_max_dias_atraso IS NULL THEN -99 ELSE vl_Bandeirado_max_dias_atraso END AS vl_Bandeirado_max_dias_atraso
    ,CASE WHEN vl_Bandeirado_min_saldo_devedor IS NULL THEN -99 ELSE vl_Bandeirado_min_saldo_devedor END AS vl_Bandeirado_min_saldo_devedor
    ,CASE WHEN vl_Bandeirado_max_saldo_devedor IS NULL THEN -99 ELSE vl_Bandeirado_max_saldo_devedor END AS vl_Bandeirado_max_saldo_devedor
    ,CASE WHEN vl_Bandeirado_total_saldo_devedor IS NULL THEN -99 ELSE vl_Bandeirado_total_saldo_devedor END AS vl_Bandeirado_total_saldo_devedor
    ,CASE WHEN vl_Emprestimo_min_score_colletion IS NULL THEN -99 ELSE vl_Emprestimo_min_score_colletion END AS vl_Emprestimo_min_score_colletion
    ,CASE WHEN vl_Emprestimo_max_score_colletion IS NULL THEN -99 ELSE vl_Emprestimo_max_score_colletion END AS vl_Emprestimo_max_score_colletion
    ,CASE WHEN vl_Emprestimo_min_dias_atraso IS NULL THEN -99 ELSE vl_Emprestimo_min_dias_atraso END AS vl_Emprestimo_min_dias_atraso
    ,CASE WHEN vl_Emprestimo_max_dias_atraso IS NULL THEN -99 ELSE vl_Emprestimo_max_dias_atraso END AS vl_Emprestimo_max_dias_atraso
    ,CASE WHEN vl_Emprestimo_min_saldo_devedor IS NULL THEN -99 ELSE vl_Emprestimo_min_saldo_devedor END AS vl_Emprestimo_min_saldo_devedor
    ,CASE WHEN vl_Emprestimo_max_saldo_devedor IS NULL THEN -99 ELSE vl_Emprestimo_max_saldo_devedor END AS vl_Emprestimo_max_saldo_devedor
    ,CASE WHEN vl_Emprestimo_total_saldo_devedor IS NULL THEN -99 ELSE vl_Emprestimo_total_saldo_devedor END AS vl_Emprestimo_total_saldo_devedor
    ,CASE WHEN vl_Outros_qnt_cartao IS NULL THEN -99 ELSE vl_Outros_qnt_cartao END AS vl_Outros_qnt_cartao
    ,CASE WHEN vl_Outros_min_limite IS NULL THEN -99 ELSE vl_Outros_min_limite END AS vl_Outros_min_limite
    ,CASE WHEN vl_Outros_max_limite IS NULL THEN -99 ELSE vl_Outros_max_limite END AS vl_Outros_max_limite
    ,CASE WHEN vl_Outros_total_limite IS NULL THEN -99 ELSE vl_Outros_total_limite END AS vl_Outros_total_limite
    ,CASE WHEN vl_Outros_min_score_colletion IS NULL THEN -99 ELSE vl_Outros_min_score_colletion END AS vl_Outros_min_score_colletion
    ,CASE WHEN vl_Outros_max_score_colletion IS NULL THEN -99 ELSE vl_Outros_max_score_colletion END AS vl_Outros_max_score_colletion
    ,CASE WHEN vl_Outros_min_dias_atraso IS NULL THEN -99 ELSE vl_Outros_min_dias_atraso END AS vl_Outros_min_dias_atraso
    ,CASE WHEN vl_Outros_max_dias_atraso IS NULL THEN -99 ELSE vl_Outros_max_dias_atraso END AS vl_Outros_max_dias_atraso
    ,CASE WHEN vl_Outros_min_saldo_devedor IS NULL THEN -99 ELSE vl_Outros_min_saldo_devedor END AS vl_Outros_min_saldo_devedor
    ,CASE WHEN vl_Outros_max_saldo_devedor IS NULL THEN -99 ELSE vl_Outros_max_saldo_devedor END AS vl_Outros_max_saldo_devedor
    ,CASE WHEN vl_Outros_total_saldo_devedor IS NULL THEN -99 ELSE vl_Outros_total_saldo_devedor END AS vl_Outros_total_saldo_devedor
    ,dt_fechamento
from df_dm
where ch_cpf_hash is not null""")

df_dm_dataprep_num_var.createOrReplaceTempView("df_dm_dataprep_num_var")





In [ ]:
# 2. Apresenta a última informação disponivel do CPF
df_dm_ultimo_num_var_final = spark.sql("""
WITH base_last as
        (SELECT ch_cpf_hash
        ,vl_qnt_cartao
        ,vl_min_score_colletion
        ,vl_max_score_colletion
        ,vl_min_dias_atraso
        ,vl_max_dias_atraso
        ,vl_min_saldo_devedor
        ,vl_max_saldo_devedor
        ,vl_total_saldo_devedor
        ,vl_min_limite
        ,vl_max_limite
        ,vl_total_limite
        ,ds_behaviour
        ,vl_valor_ultimo_movimento
        ,vl_saldo_dm_conta
        ,id_cliente_nivel
        ,id_ciclo
        ,vl_dias_atraso_dmcred
        ,vl_saldo_devedor_dmcred
        ,vl_qnt_compras_dmpag
        ,vl_total_compra
        ,vl_dias_atraso_dmpag
        ,vl_saldo_devedor_dmpag
        ,vl_Farmacia_qnt_cartao
        ,vl_Farmacia_min_limite
        ,vl_Farmacia_max_limite
        ,vl_Farmacia_total_limite
        ,vl_Farmacia_min_score_colletion
        ,vl_Farmacia_max_score_colletion
        ,vl_Farmacia_min_dias_atraso
        ,vl_Farmacia_max_dias_atraso
        ,vl_Farmacia_min_saldo_devedor
        ,vl_Farmacia_max_saldo_devedor
        ,vl_Farmacia_total_saldo_devedor
        ,vl_Mercado_qnt_cartao
        ,vl_Mercado_min_limite
        ,vl_Mercado_max_limite
        ,vl_Mercado_total_limite
        ,vl_Mercado_min_score_colletion
        ,vl_Mercado_max_score_colletion
        ,vl_Mercado_min_dias_atraso
        ,vl_Mercado_max_dias_atraso
        ,vl_Mercado_min_saldo_devedor
        ,vl_Mercado_max_saldo_devedor
        ,vl_Mercado_total_saldo_devedor
        ,vl_Construcao_qnt_cartao
        ,vl_Construcao_min_limite
        ,vl_Construcao_max_limite
        ,vl_Construcao_total_limite
        ,vl_Construcao_min_score_colletion
        ,vl_Construcao_max_score_colletion
        ,vl_Construcao_min_dias_atraso
        ,vl_Construcao_max_dias_atraso
        ,vl_Construcao_min_saldo_devedor
        ,vl_Construcao_max_saldo_devedor
        ,vl_Construcao_total_saldo_devedor
        ,vl_Bandeirado_qnt_cartao
        ,vl_Bandeirado_min_limite
        ,vl_Bandeirado_max_limite
        ,vl_Bandeirado_total_limite
        ,vl_Bandeirado_min_score_colletion
        ,vl_Bandeirado_max_score_colletion
        ,vl_Bandeirado_min_dias_atraso
        ,vl_Bandeirado_max_dias_atraso
        ,vl_Bandeirado_min_saldo_devedor
        ,vl_Bandeirado_max_saldo_devedor
        ,vl_Bandeirado_total_saldo_devedor
        ,vl_Emprestimo_min_score_colletion
        ,vl_Emprestimo_max_score_colletion
        ,vl_Emprestimo_min_dias_atraso
        ,vl_Emprestimo_max_dias_atraso
        ,vl_Emprestimo_min_saldo_devedor
        ,vl_Emprestimo_max_saldo_devedor
        ,vl_Emprestimo_total_saldo_devedor
        ,vl_Outros_qnt_cartao
        ,vl_Outros_min_limite
        ,vl_Outros_max_limite
        ,vl_Outros_total_limite
        ,vl_Outros_min_score_colletion
        ,vl_Outros_max_score_colletion
        ,vl_Outros_min_dias_atraso
        ,vl_Outros_max_dias_atraso
        ,vl_Outros_min_saldo_devedor
        ,vl_Outros_max_saldo_devedor
        ,vl_Outros_total_saldo_devedor
        ,dt_fechamento
        ,ROW_NUMBER() OVER(PARTITION BY ch_cpf_hash ORDER BY dt_fechamento desc) as num_ultimo
        FROM df_dm_dataprep_num_var)
SELECT ch_cpf_hash
    ,vl_qnt_cartao as vl_qnt_cartao_ultimo
    ,vl_min_score_colletion as vl_min_score_colletion_ultimo
    ,vl_max_score_colletion as vl_max_score_colletion_ultimo
    ,vl_min_dias_atraso as vl_min_dias_atraso_ultimo
    ,vl_max_dias_atraso as vl_max_dias_atraso_ultimo
    ,vl_min_saldo_devedor as vl_min_saldo_devedor_ultimo
    ,vl_max_saldo_devedor as vl_max_saldo_devedor_ultimo
    ,vl_total_saldo_devedor as vl_total_saldo_devedor_ultimo
    ,vl_min_limite as vl_min_limite_ultimo
    ,vl_max_limite as vl_max_limite_ultimo
    ,vl_total_limite as vl_total_limite_ultimo
    ,ds_behaviour as ds_behaviour_ultimo
    ,vl_valor_ultimo_movimento as vl_valor_ultimo_movimento_ultimo
    ,vl_saldo_dm_conta as vl_saldo_dm_conta_ultimo
    ,id_cliente_nivel as id_cliente_nivel_ultimo
    ,id_ciclo as id_ciclo_ultimo
    ,vl_dias_atraso_dmcred as vl_dias_atraso_dmcred_ultimo
    ,vl_saldo_devedor_dmcred as vl_saldo_devedor_dmcred_ultimo
    ,vl_qnt_compras_dmpag as vl_qnt_compras_dmpag_ultimo
    ,vl_total_compra as vl_total_compra_ultimo
    ,vl_dias_atraso_dmpag as vl_dias_atraso_dmpag_ultimo
    ,vl_saldo_devedor_dmpag as vl_saldo_devedor_dmpag_ultimo
    ,vl_Farmacia_qnt_cartao as vl_Farmacia_qnt_cartao_ultimo
    ,vl_Farmacia_min_limite as vl_Farmacia_min_limite_ultimo
    ,vl_Farmacia_max_limite as vl_Farmacia_max_limite_ultimo
    ,vl_Farmacia_total_limite as vl_Farmacia_total_limite_ultimo
    ,vl_Farmacia_min_score_colletion as vl_Farmacia_min_score_colletion_ultimo
    ,vl_Farmacia_max_score_colletion as vl_Farmacia_max_score_colletion_ultimo
    ,vl_Farmacia_min_dias_atraso as vl_Farmacia_min_dias_atraso_ultimo
    ,vl_Farmacia_max_dias_atraso as vl_Farmacia_max_dias_atraso_ultimo
    ,vl_Farmacia_min_saldo_devedor as vl_Farmacia_min_saldo_devedor_ultimo
    ,vl_Farmacia_max_saldo_devedor as vl_Farmacia_max_saldo_devedor_ultimo
    ,vl_Farmacia_total_saldo_devedor as vl_Farmacia_total_saldo_devedor_ultimo
    ,vl_Mercado_qnt_cartao as vl_Mercado_qnt_cartao_ultimo
    ,vl_Mercado_min_limite as vl_Mercado_min_limite_ultimo
    ,vl_Mercado_max_limite as vl_Mercado_max_limite_ultimo
    ,vl_Mercado_total_limite as vl_Mercado_total_limite_ultimo
    ,vl_Mercado_min_score_colletion as vl_Mercado_min_score_colletion_ultimo
    ,vl_Mercado_max_score_colletion as vl_Mercado_max_score_colletion_ultimo
    ,vl_Mercado_min_dias_atraso as vl_Mercado_min_dias_atraso_ultimo
    ,vl_Mercado_max_dias_atraso as vl_Mercado_max_dias_atraso_ultimo
    ,vl_Mercado_min_saldo_devedor as vl_Mercado_min_saldo_devedor_ultimo
    ,vl_Mercado_max_saldo_devedor as vl_Mercado_max_saldo_devedor_ultimo
    ,vl_Mercado_total_saldo_devedor as vl_Mercado_total_saldo_devedor_ultimo
    ,vl_Construcao_qnt_cartao as vl_Construcao_qnt_cartao_ultimo
    ,vl_Construcao_min_limite as vl_Construcao_min_limite_ultimo
    ,vl_Construcao_max_limite as vl_Construcao_max_limite_ultimo
    ,vl_Construcao_total_limite as vl_Construcao_total_limite_ultimo
    ,vl_Construcao_min_score_colletion as vl_Construcao_min_score_colletion_ultimo
    ,vl_Construcao_max_score_colletion as vl_Construcao_max_score_colletion_ultimo
    ,vl_Construcao_min_dias_atraso as vl_Construcao_min_dias_atraso_ultimo
    ,vl_Construcao_max_dias_atraso as vl_Construcao_max_dias_atraso_ultimo
    ,vl_Construcao_min_saldo_devedor as vl_Construcao_min_saldo_devedor_ultimo
    ,vl_Construcao_max_saldo_devedor as vl_Construcao_max_saldo_devedor_ultimo
    ,vl_Construcao_total_saldo_devedor as vl_Construcao_total_saldo_devedor_ultimo
    ,vl_Bandeirado_qnt_cartao as vl_Bandeirado_qnt_cartao_ultimo
    ,vl_Bandeirado_min_limite as vl_Bandeirado_min_limite_ultimo
    ,vl_Bandeirado_max_limite as vl_Bandeirado_max_limite_ultimo
    ,vl_Bandeirado_total_limite as vl_Bandeirado_total_limite_ultimo
    ,vl_Bandeirado_min_score_colletion as vl_Bandeirado_min_score_colletion_ultimo
    ,vl_Bandeirado_max_score_colletion as vl_Bandeirado_max_score_colletion_ultimo
    ,vl_Bandeirado_min_dias_atraso as vl_Bandeirado_min_dias_atraso_ultimo
    ,vl_Bandeirado_max_dias_atraso as vl_Bandeirado_max_dias_atraso_ultimo
    ,vl_Bandeirado_min_saldo_devedor as vl_Bandeirado_min_saldo_devedor_ultimo
    ,vl_Bandeirado_max_saldo_devedor as vl_Bandeirado_max_saldo_devedor_ultimo
    ,vl_Bandeirado_total_saldo_devedor as vl_Bandeirado_total_saldo_devedor_ultimo
    ,vl_Emprestimo_min_score_colletion as vl_Emprestimo_min_score_colletion_ultimo
    ,vl_Emprestimo_max_score_colletion as vl_Emprestimo_max_score_colletion_ultimo
    ,vl_Emprestimo_min_dias_atraso as vl_Emprestimo_min_dias_atraso_ultimo
    ,vl_Emprestimo_max_dias_atraso as vl_Emprestimo_max_dias_atraso_ultimo
    ,vl_Emprestimo_min_saldo_devedor as vl_Emprestimo_min_saldo_devedor_ultimo
    ,vl_Emprestimo_max_saldo_devedor as vl_Emprestimo_max_saldo_devedor_ultimo
    ,vl_Emprestimo_total_saldo_devedor as vl_Emprestimo_total_saldo_devedor_ultimo
    ,vl_Outros_qnt_cartao as vl_Outros_qnt_cartao_ultimo
    ,vl_Outros_min_limite as vl_Outros_min_limite_ultimo
    ,vl_Outros_max_limite as vl_Outros_max_limite_ultimo
    ,vl_Outros_total_limite as vl_Outros_total_limite_ultimo
    ,vl_Outros_min_score_colletion as vl_Outros_min_score_colletion_ultimo
    ,vl_Outros_max_score_colletion as vl_Outros_max_score_colletion_ultimo
    ,vl_Outros_min_dias_atraso as vl_Outros_min_dias_atraso_ultimo
    ,vl_Outros_max_dias_atraso as vl_Outros_max_dias_atraso_ultimo
    ,vl_Outros_min_saldo_devedor as vl_Outros_min_saldo_devedor_ultimo
    ,vl_Outros_max_saldo_devedor as vl_Outros_max_saldo_devedor_ultimo
    ,vl_Outros_total_saldo_devedor as vl_Outros_total_saldo_devedor_ultimo
    ,dt_fechamento as dt_fechamento_ultimo
FROM base_last
where num_ultimo = 1 """)

df_dm_ultimo_num_var_final.createOrReplaceTempView("df_dm_ultimo_num_var_final")




In [ ]:
# 3. Apresenta a primeira informação disponivel do CPF
df_dm_primeiro_num_var_final = spark.sql("""
WITH base_last as
        (SELECT ch_cpf_hash
        ,vl_qnt_cartao
        ,vl_min_score_colletion
        ,vl_max_score_colletion
        ,vl_min_dias_atraso
        ,vl_max_dias_atraso
        ,vl_min_saldo_devedor
        ,vl_max_saldo_devedor
        ,vl_total_saldo_devedor
        ,vl_min_limite
        ,vl_max_limite
        ,vl_total_limite
        ,ds_behaviour
        ,vl_valor_ultimo_movimento
        ,vl_saldo_dm_conta
        ,id_cliente_nivel
        ,id_ciclo
        ,vl_dias_atraso_dmcred
        ,vl_saldo_devedor_dmcred
        ,vl_qnt_compras_dmpag
        ,vl_total_compra
        ,vl_dias_atraso_dmpag
        ,vl_saldo_devedor_dmpag
        ,vl_Farmacia_qnt_cartao
        ,vl_Farmacia_min_limite
        ,vl_Farmacia_max_limite
        ,vl_Farmacia_total_limite
        ,vl_Farmacia_min_score_colletion
        ,vl_Farmacia_max_score_colletion
        ,vl_Farmacia_min_dias_atraso
        ,vl_Farmacia_max_dias_atraso
        ,vl_Farmacia_min_saldo_devedor
        ,vl_Farmacia_max_saldo_devedor
        ,vl_Farmacia_total_saldo_devedor
        ,vl_Mercado_qnt_cartao
        ,vl_Mercado_min_limite
        ,vl_Mercado_max_limite
        ,vl_Mercado_total_limite
        ,vl_Mercado_min_score_colletion
        ,vl_Mercado_max_score_colletion
        ,vl_Mercado_min_dias_atraso
        ,vl_Mercado_max_dias_atraso
        ,vl_Mercado_min_saldo_devedor
        ,vl_Mercado_max_saldo_devedor
        ,vl_Mercado_total_saldo_devedor
        ,vl_Construcao_qnt_cartao
        ,vl_Construcao_min_limite
        ,vl_Construcao_max_limite
        ,vl_Construcao_total_limite
        ,vl_Construcao_min_score_colletion
        ,vl_Construcao_max_score_colletion
        ,vl_Construcao_min_dias_atraso
        ,vl_Construcao_max_dias_atraso
        ,vl_Construcao_min_saldo_devedor
        ,vl_Construcao_max_saldo_devedor
        ,vl_Construcao_total_saldo_devedor
        ,vl_Bandeirado_qnt_cartao
        ,vl_Bandeirado_min_limite
        ,vl_Bandeirado_max_limite
        ,vl_Bandeirado_total_limite
        ,vl_Bandeirado_min_score_colletion
        ,vl_Bandeirado_max_score_colletion
        ,vl_Bandeirado_min_dias_atraso
        ,vl_Bandeirado_max_dias_atraso
        ,vl_Bandeirado_min_saldo_devedor
        ,vl_Bandeirado_max_saldo_devedor
        ,vl_Bandeirado_total_saldo_devedor
        ,vl_Emprestimo_min_score_colletion
        ,vl_Emprestimo_max_score_colletion
        ,vl_Emprestimo_min_dias_atraso
        ,vl_Emprestimo_max_dias_atraso
        ,vl_Emprestimo_min_saldo_devedor
        ,vl_Emprestimo_max_saldo_devedor
        ,vl_Emprestimo_total_saldo_devedor
        ,vl_Outros_qnt_cartao
        ,vl_Outros_min_limite
        ,vl_Outros_max_limite
        ,vl_Outros_total_limite
        ,vl_Outros_min_score_colletion
        ,vl_Outros_max_score_colletion
        ,vl_Outros_min_dias_atraso
        ,vl_Outros_max_dias_atraso
        ,vl_Outros_min_saldo_devedor
        ,vl_Outros_max_saldo_devedor
        ,vl_Outros_total_saldo_devedor
        ,dt_fechamento
        ,ROW_NUMBER() OVER(PARTITION BY ch_cpf_hash ORDER BY dt_fechamento asc) as num_primeiro
        FROM df_dm_dataprep_num_var)
SELECT ch_cpf_hash
    ,vl_qnt_cartao as vl_qnt_cartao_primeiro
    ,vl_min_score_colletion as vl_min_score_colletion_primeiro
    ,vl_max_score_colletion as vl_max_score_colletion_primeiro
    ,vl_min_dias_atraso as vl_min_dias_atraso_primeiro
    ,vl_max_dias_atraso as vl_max_dias_atraso_primeiro
    ,vl_min_saldo_devedor as vl_min_saldo_devedor_primeiro
    ,vl_max_saldo_devedor as vl_max_saldo_devedor_primeiro
    ,vl_total_saldo_devedor as vl_total_saldo_devedor_primeiro
    ,vl_min_limite as vl_min_limite_primeiro
    ,vl_max_limite as vl_max_limite_primeiro
    ,vl_total_limite as vl_total_limite_primeiro
    ,ds_behaviour as ds_behaviour_primeiro
    ,vl_valor_ultimo_movimento as vl_valor_ultimo_movimento_primeiro
    ,vl_saldo_dm_conta as vl_saldo_dm_conta_primeiro
    ,id_cliente_nivel as id_cliente_nivel_primeiro
    ,id_ciclo as id_ciclo_primeiro
    ,vl_dias_atraso_dmcred as vl_dias_atraso_dmcred_primeiro
    ,vl_saldo_devedor_dmcred as vl_saldo_devedor_dmcred_primeiro
    ,vl_qnt_compras_dmpag as vl_qnt_compras_dmpag_primeiro
    ,vl_total_compra as vl_total_compra_primeiro
    ,vl_dias_atraso_dmpag as vl_dias_atraso_dmpag_primeiro
    ,vl_saldo_devedor_dmpag as vl_saldo_devedor_dmpag_primeiro
    ,vl_Farmacia_qnt_cartao as vl_Farmacia_qnt_cartao_primeiro
    ,vl_Farmacia_min_limite as vl_Farmacia_min_limite_primeiro
    ,vl_Farmacia_max_limite as vl_Farmacia_max_limite_primeiro
    ,vl_Farmacia_total_limite as vl_Farmacia_total_limite_primeiro
    ,vl_Farmacia_min_score_colletion as vl_Farmacia_min_score_colletion_primeiro
    ,vl_Farmacia_max_score_colletion as vl_Farmacia_max_score_colletion_primeiro
    ,vl_Farmacia_min_dias_atraso as vl_Farmacia_min_dias_atraso_primeiro
    ,vl_Farmacia_max_dias_atraso as vl_Farmacia_max_dias_atraso_primeiro
    ,vl_Farmacia_min_saldo_devedor as vl_Farmacia_min_saldo_devedor_primeiro
    ,vl_Farmacia_max_saldo_devedor as vl_Farmacia_max_saldo_devedor_primeiro
    ,vl_Farmacia_total_saldo_devedor as vl_Farmacia_total_saldo_devedor_primeiro
    ,vl_Mercado_qnt_cartao as vl_Mercado_qnt_cartao_primeiro
    ,vl_Mercado_min_limite as vl_Mercado_min_limite_primeiro
    ,vl_Mercado_max_limite as vl_Mercado_max_limite_primeiro
    ,vl_Mercado_total_limite as vl_Mercado_total_limite_primeiro
    ,vl_Mercado_min_score_colletion as vl_Mercado_min_score_colletion_primeiro
    ,vl_Mercado_max_score_colletion as vl_Mercado_max_score_colletion_primeiro
    ,vl_Mercado_min_dias_atraso as vl_Mercado_min_dias_atraso_primeiro
    ,vl_Mercado_max_dias_atraso as vl_Mercado_max_dias_atraso_primeiro
    ,vl_Mercado_min_saldo_devedor as vl_Mercado_min_saldo_devedor_primeiro
    ,vl_Mercado_max_saldo_devedor as vl_Mercado_max_saldo_devedor_primeiro
    ,vl_Mercado_total_saldo_devedor as vl_Mercado_total_saldo_devedor_primeiro
    ,vl_Construcao_qnt_cartao as vl_Construcao_qnt_cartao_primeiro
    ,vl_Construcao_min_limite as vl_Construcao_min_limite_primeiro
    ,vl_Construcao_max_limite as vl_Construcao_max_limite_primeiro
    ,vl_Construcao_total_limite as vl_Construcao_total_limite_primeiro
    ,vl_Construcao_min_score_colletion as vl_Construcao_min_score_colletion_primeiro
    ,vl_Construcao_max_score_colletion as vl_Construcao_max_score_colletion_primeiro
    ,vl_Construcao_min_dias_atraso as vl_Construcao_min_dias_atraso_primeiro
    ,vl_Construcao_max_dias_atraso as vl_Construcao_max_dias_atraso_primeiro
    ,vl_Construcao_min_saldo_devedor as vl_Construcao_min_saldo_devedor_primeiro
    ,vl_Construcao_max_saldo_devedor as vl_Construcao_max_saldo_devedor_primeiro
    ,vl_Construcao_total_saldo_devedor as vl_Construcao_total_saldo_devedor_primeiro
    ,vl_Bandeirado_qnt_cartao as vl_Bandeirado_qnt_cartao_primeiro
    ,vl_Bandeirado_min_limite as vl_Bandeirado_min_limite_primeiro
    ,vl_Bandeirado_max_limite as vl_Bandeirado_max_limite_primeiro
    ,vl_Bandeirado_total_limite as vl_Bandeirado_total_limite_primeiro
    ,vl_Bandeirado_min_score_colletion as vl_Bandeirado_min_score_colletion_primeiro
    ,vl_Bandeirado_max_score_colletion as vl_Bandeirado_max_score_colletion_primeiro
    ,vl_Bandeirado_min_dias_atraso as vl_Bandeirado_min_dias_atraso_primeiro
    ,vl_Bandeirado_max_dias_atraso as vl_Bandeirado_max_dias_atraso_primeiro
    ,vl_Bandeirado_min_saldo_devedor as vl_Bandeirado_min_saldo_devedor_primeiro
    ,vl_Bandeirado_max_saldo_devedor as vl_Bandeirado_max_saldo_devedor_primeiro
    ,vl_Bandeirado_total_saldo_devedor as vl_Bandeirado_total_saldo_devedor_primeiro
    ,vl_Emprestimo_min_score_colletion as vl_Emprestimo_min_score_colletion_primeiro
    ,vl_Emprestimo_max_score_colletion as vl_Emprestimo_max_score_colletion_primeiro
    ,vl_Emprestimo_min_dias_atraso as vl_Emprestimo_min_dias_atraso_primeiro
    ,vl_Emprestimo_max_dias_atraso as vl_Emprestimo_max_dias_atraso_primeiro
    ,vl_Emprestimo_min_saldo_devedor as vl_Emprestimo_min_saldo_devedor_primeiro
    ,vl_Emprestimo_max_saldo_devedor as vl_Emprestimo_max_saldo_devedor_primeiro
    ,vl_Emprestimo_total_saldo_devedor as vl_Emprestimo_total_saldo_devedor_primeiro
    ,vl_Outros_qnt_cartao as vl_Outros_qnt_cartao_primeiro
    ,vl_Outros_min_limite as vl_Outros_min_limite_primeiro
    ,vl_Outros_max_limite as vl_Outros_max_limite_primeiro
    ,vl_Outros_total_limite as vl_Outros_total_limite_primeiro
    ,vl_Outros_min_score_colletion as vl_Outros_min_score_colletion_primeiro
    ,vl_Outros_max_score_colletion as vl_Outros_max_score_colletion_primeiro
    ,vl_Outros_min_dias_atraso as vl_Outros_min_dias_atraso_primeiro
    ,vl_Outros_max_dias_atraso as vl_Outros_max_dias_atraso_primeiro
    ,vl_Outros_min_saldo_devedor as vl_Outros_min_saldo_devedor_primeiro
    ,vl_Outros_max_saldo_devedor as vl_Outros_max_saldo_devedor_primeiro
    ,vl_Outros_total_saldo_devedor as vl_Outros_total_saldo_devedor_primeiro
    ,dt_fechamento as dt_fechamento_primeiro
FROM base_last
where num_primeiro = 1 """)

df_dm_primeiro_num_var_final.createOrReplaceTempView("df_dm_primeiro_num_var_final")






In [ ]:
# 4. Calcula Máximos, Mínimos, Decis e Quartis das variáveis numéricas
df_dm_dataprep_num_var_final = spark.sql("""SELECT ch_cpf_hash
    -- Máximos
    ,MAX(vl_qnt_cartao) AS vl_qnt_cartao_var_max
    ,MAX(vl_min_score_colletion) AS vl_min_score_colletion_var_max
    ,MAX(vl_max_score_colletion) AS vl_max_score_colletion_var_max
    ,MAX(vl_min_dias_atraso) AS vl_min_dias_atraso_var_max
    ,MAX(vl_max_dias_atraso) AS vl_max_dias_atraso_var_max
    ,MAX(vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_max
    ,MAX(vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_max
    ,MAX(vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_max
    ,MAX(vl_min_limite) AS vl_min_limite_var_max
    ,MAX(vl_max_limite) AS vl_max_limite_var_max
    ,MAX(vl_total_limite) AS vl_total_limite_var_max
    ,MAX(ds_behaviour) AS ds_behaviour_var_max
    ,MAX(vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_max
    ,MAX(vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_max
    ,MAX(id_cliente_nivel) AS id_cliente_nivel_var_max
    ,MAX(id_ciclo) AS id_ciclo_var_max
    ,MAX(vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_max
    ,MAX(vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_max
    ,MAX(vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_max
    ,MAX(vl_total_compra) AS vl_total_compra_var_max
    ,MAX(vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_max
    ,MAX(vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_max
    ,MAX(vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_max
    ,MAX(vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_max
    ,MAX(vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_max
    ,MAX(vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_max
    ,MAX(vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_max
    ,MAX(vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_max
    ,MAX(vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_max
    ,MAX(vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_max
    ,MAX(vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_max
    ,MAX(vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_max
    ,MAX(vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_max
    ,MAX(vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_max
    ,MAX(vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_max
    ,MAX(vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_max
    ,MAX(vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_max
    ,MAX(vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_max
    ,MAX(vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_max
    ,MAX(vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_max
    ,MAX(vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_max
    ,MAX(vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_max
    ,MAX(vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_max
    ,MAX(vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_max
    ,MAX(vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_max
    ,MAX(vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_max
    ,MAX(vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_max
    ,MAX(vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_max
    ,MAX(vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_max
    ,MAX(vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_max
    ,MAX(vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_max
    ,MAX(vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_max
    ,MAX(vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_max
    ,MAX(vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_max
    ,MAX(vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_max
    ,MAX(vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_max
    ,MAX(vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_max
    ,MAX(vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_max
    ,MAX(vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_max
    ,MAX(vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_max
    ,MAX(vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_max
    ,MAX(vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_max
    ,MAX(vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_max
    ,MAX(vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_max
    ,MAX(vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_max
    ,MAX(vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_max
    ,MAX(vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_max
    ,MAX(vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_max
    ,MAX(vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_max
    ,MAX(vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_max
    ,MAX(vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_max
    ,MAX(vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_max
    ,MAX(vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_max
    ,MAX(vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_max
    ,MAX(vl_Outros_min_limite) AS vl_Outros_min_limite_var_max
    ,MAX(vl_Outros_max_limite) AS vl_Outros_max_limite_var_max
    ,MAX(vl_Outros_total_limite) AS vl_Outros_total_limite_var_max
    ,MAX(vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_max
    ,MAX(vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_max
    ,MAX(vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_max
    ,MAX(vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_max
    ,MAX(vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_max
    ,MAX(vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_max
    ,MAX(vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_max
    -- Mínimos
    ,MIN(vl_qnt_cartao) AS vl_qnt_cartao_var_min
    ,MIN(vl_min_score_colletion) AS vl_min_score_colletion_var_min
    ,MIN(vl_max_score_colletion) AS vl_max_score_colletion_var_min
    ,MIN(vl_min_dias_atraso) AS vl_min_dias_atraso_var_min
    ,MIN(vl_max_dias_atraso) AS vl_max_dias_atraso_var_min
    ,MIN(vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_min
    ,MIN(vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_min
    ,MIN(vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_min
    ,MIN(vl_min_limite) AS vl_min_limite_var_min
    ,MIN(vl_max_limite) AS vl_max_limite_var_min
    ,MIN(vl_total_limite) AS vl_total_limite_var_min
    ,MIN(ds_behaviour) AS ds_behaviour_var_min
    ,MIN(vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_min
    ,MIN(vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_min
    ,MIN(id_cliente_nivel) AS id_cliente_nivel_var_min
    ,MIN(id_ciclo) AS id_ciclo_var_min
    ,MIN(vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_min
    ,MIN(vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_min
    ,MIN(vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_min
    ,MIN(vl_total_compra) AS vl_total_compra_var_min
    ,MIN(vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_min
    ,MIN(vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_min
    ,MIN(vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_min
    ,MIN(vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_min
    ,MIN(vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_min
    ,MIN(vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_min
    ,MIN(vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_min
    ,MIN(vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_min
    ,MIN(vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_min
    ,MIN(vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_min
    ,MIN(vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_min
    ,MIN(vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_min
    ,MIN(vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_min
    ,MIN(vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_min
    ,MIN(vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_min
    ,MIN(vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_min
    ,MIN(vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_min
    ,MIN(vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_min
    ,MIN(vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_min
    ,MIN(vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_min
    ,MIN(vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_min
    ,MIN(vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_min
    ,MIN(vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_min
    ,MIN(vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_min
    ,MIN(vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_min
    ,MIN(vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_min
    ,MIN(vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_min
    ,MIN(vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_min
    ,MIN(vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_min
    ,MIN(vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_min
    ,MIN(vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_min
    ,MIN(vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_min
    ,MIN(vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_min
    ,MIN(vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_min
    ,MIN(vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_min
    ,MIN(vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_min
    ,MIN(vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_min
    ,MIN(vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_min
    ,MIN(vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_min
    ,MIN(vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_min
    ,MIN(vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_min
    ,MIN(vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_min
    ,MIN(vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_min
    ,MIN(vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_min
    ,MIN(vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_min
    ,MIN(vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_min
    ,MIN(vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_min
    ,MIN(vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_min
    ,MIN(vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_min
    ,MIN(vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_min
    ,MIN(vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_min
    ,MIN(vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_min
    ,MIN(vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_min
    ,MIN(vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_min
    ,MIN(vl_Outros_min_limite) AS vl_Outros_min_limite_var_min
    ,MIN(vl_Outros_max_limite) AS vl_Outros_max_limite_var_min
    ,MIN(vl_Outros_total_limite) AS vl_Outros_total_limite_var_min
    ,MIN(vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_min
    ,MIN(vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_min
    ,MIN(vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_min
    ,MIN(vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_min
    ,MIN(vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_min
    ,MIN(vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_min
    ,MIN(vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_min
    -- Médias
    ,AVG(vl_qnt_cartao) AS vl_qnt_cartao_var_avg
    ,AVG(vl_min_score_colletion) AS vl_min_score_colletion_var_avg
    ,AVG(vl_max_score_colletion) AS vl_max_score_colletion_var_avg
    ,AVG(vl_min_dias_atraso) AS vl_min_dias_atraso_var_avg
    ,AVG(vl_max_dias_atraso) AS vl_max_dias_atraso_var_avg
    ,AVG(vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_avg
    ,AVG(vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_avg
    ,AVG(vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_avg
    ,AVG(vl_min_limite) AS vl_min_limite_var_avg
    ,AVG(vl_max_limite) AS vl_max_limite_var_avg
    ,AVG(vl_total_limite) AS vl_total_limite_var_avg
    ,AVG(ds_behaviour) AS ds_behaviour_var_avg
    ,AVG(vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_avg
    ,AVG(vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_avg
    ,AVG(id_cliente_nivel) AS id_cliente_nivel_var_avg
    ,AVG(id_ciclo) AS id_ciclo_var_avg
    ,AVG(vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_avg
    ,AVG(vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_avg
    ,AVG(vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_avg
    ,AVG(vl_total_compra) AS vl_total_compra_var_avg
    ,AVG(vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_avg
    ,AVG(vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_avg
    ,AVG(vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_avg
    ,AVG(vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_avg
    ,AVG(vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_avg
    ,AVG(vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_avg
    ,AVG(vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_avg
    ,AVG(vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_avg
    ,AVG(vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_avg
    ,AVG(vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_avg
    ,AVG(vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_avg
    ,AVG(vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_avg
    ,AVG(vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_avg
    ,AVG(vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_avg
    ,AVG(vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_avg
    ,AVG(vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_avg
    ,AVG(vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_avg
    ,AVG(vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_avg
    ,AVG(vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_avg
    ,AVG(vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_avg
    ,AVG(vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_avg
    ,AVG(vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_avg
    ,AVG(vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_avg
    ,AVG(vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_avg
    ,AVG(vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_avg
    ,AVG(vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_avg
    ,AVG(vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_avg
    ,AVG(vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_avg
    ,AVG(vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_avg
    ,AVG(vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_avg
    ,AVG(vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_avg
    ,AVG(vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_avg
    ,AVG(vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_avg
    ,AVG(vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_avg
    ,AVG(vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_avg
    ,AVG(vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_avg
    ,AVG(vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_avg
    ,AVG(vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_avg
    ,AVG(vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_avg
    ,AVG(vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_avg
    ,AVG(vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_avg
    ,AVG(vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_avg
    ,AVG(vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_avg
    ,AVG(vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_avg
    ,AVG(vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_avg
    ,AVG(vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_avg
    ,AVG(vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_avg
    ,AVG(vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_avg
    ,AVG(vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_avg
    ,AVG(vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_avg
    ,AVG(vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_avg
    ,AVG(vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_avg
    ,AVG(vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_avg
    ,AVG(vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_avg
    ,AVG(vl_Outros_min_limite) AS vl_Outros_min_limite_var_avg
    ,AVG(vl_Outros_max_limite) AS vl_Outros_max_limite_var_avg
    ,AVG(vl_Outros_total_limite) AS vl_Outros_total_limite_var_avg
    ,AVG(vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_avg
    ,AVG(vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_avg
    ,AVG(vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_avg
    ,AVG(vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_avg
    ,AVG(vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_avg
    ,AVG(vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_avg
    ,AVG(vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_avg
    -- Decis e quartis
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.10) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_1_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.20) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_2_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.30) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_3_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.40) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_4_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_5_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.60) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_6_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.70) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_7_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_8_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_9_decil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_1_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_qnt_cartao) AS vl_qnt_cartao_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_min_score_colletion) AS vl_min_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_max_score_colletion) AS vl_max_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_min_dias_atraso) AS vl_min_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_max_dias_atraso) AS vl_max_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_min_saldo_devedor) AS vl_min_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_max_saldo_devedor) AS vl_max_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_total_saldo_devedor) AS vl_total_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_min_limite) AS vl_min_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_max_limite) AS vl_max_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_total_limite) AS vl_total_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY ds_behaviour) AS ds_behaviour_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_valor_ultimo_movimento) AS vl_valor_ultimo_movimento_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_saldo_dm_conta) AS vl_saldo_dm_conta_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY id_cliente_nivel) AS id_cliente_nivel_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY id_ciclo) AS id_ciclo_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_dias_atraso_dmcred) AS vl_dias_atraso_dmcred_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmcred) AS vl_saldo_devedor_dmcred_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_qnt_compras_dmpag) AS vl_qnt_compras_dmpag_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_total_compra) AS vl_total_compra_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_dias_atraso_dmpag) AS vl_dias_atraso_dmpag_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_saldo_devedor_dmpag) AS vl_saldo_devedor_dmpag_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_qnt_cartao) AS vl_Farmacia_qnt_cartao_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_min_limite) AS vl_Farmacia_min_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_max_limite) AS vl_Farmacia_max_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_total_limite) AS vl_Farmacia_total_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_min_score_colletion) AS vl_Farmacia_min_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_max_score_colletion) AS vl_Farmacia_max_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_min_dias_atraso) AS vl_Farmacia_min_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_max_dias_atraso) AS vl_Farmacia_max_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_min_saldo_devedor) AS vl_Farmacia_min_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_max_saldo_devedor) AS vl_Farmacia_max_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Farmacia_total_saldo_devedor) AS vl_Farmacia_total_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_qnt_cartao) AS vl_Mercado_qnt_cartao_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_min_limite) AS vl_Mercado_min_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_max_limite) AS vl_Mercado_max_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_total_limite) AS vl_Mercado_total_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_min_score_colletion) AS vl_Mercado_min_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_max_score_colletion) AS vl_Mercado_max_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_min_dias_atraso) AS vl_Mercado_min_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_max_dias_atraso) AS vl_Mercado_max_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_min_saldo_devedor) AS vl_Mercado_min_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_max_saldo_devedor) AS vl_Mercado_max_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Mercado_total_saldo_devedor) AS vl_Mercado_total_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_qnt_cartao) AS vl_Construcao_qnt_cartao_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_min_limite) AS vl_Construcao_min_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_max_limite) AS vl_Construcao_max_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_total_limite) AS vl_Construcao_total_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_min_score_colletion) AS vl_Construcao_min_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_max_score_colletion) AS vl_Construcao_max_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_min_dias_atraso) AS vl_Construcao_min_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_max_dias_atraso) AS vl_Construcao_max_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_min_saldo_devedor) AS vl_Construcao_min_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_max_saldo_devedor) AS vl_Construcao_max_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Construcao_total_saldo_devedor) AS vl_Construcao_total_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_qnt_cartao) AS vl_Bandeirado_qnt_cartao_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_min_limite) AS vl_Bandeirado_min_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_max_limite) AS vl_Bandeirado_max_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_total_limite) AS vl_Bandeirado_total_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_min_score_colletion) AS vl_Bandeirado_min_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_max_score_colletion) AS vl_Bandeirado_max_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_min_dias_atraso) AS vl_Bandeirado_min_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_max_dias_atraso) AS vl_Bandeirado_max_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_min_saldo_devedor) AS vl_Bandeirado_min_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_max_saldo_devedor) AS vl_Bandeirado_max_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Bandeirado_total_saldo_devedor) AS vl_Bandeirado_total_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Emprestimo_min_score_colletion) AS vl_Emprestimo_min_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Emprestimo_max_score_colletion) AS vl_Emprestimo_max_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Emprestimo_min_dias_atraso) AS vl_Emprestimo_min_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Emprestimo_max_dias_atraso) AS vl_Emprestimo_max_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Emprestimo_min_saldo_devedor) AS vl_Emprestimo_min_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Emprestimo_max_saldo_devedor) AS vl_Emprestimo_max_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Emprestimo_total_saldo_devedor) AS vl_Emprestimo_total_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_qnt_cartao) AS vl_Outros_qnt_cartao_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_min_limite) AS vl_Outros_min_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_max_limite) AS vl_Outros_max_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_total_limite) AS vl_Outros_total_limite_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_min_score_colletion) AS vl_Outros_min_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_max_score_colletion) AS vl_Outros_max_score_colletion_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_min_dias_atraso) AS vl_Outros_min_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_max_dias_atraso) AS vl_Outros_max_dias_atraso_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_min_saldo_devedor) AS vl_Outros_min_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_max_saldo_devedor) AS vl_Outros_max_saldo_devedor_var_3_quartil
    ,PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY vl_Outros_total_saldo_devedor) AS vl_Outros_total_saldo_devedor_var_3_quartil
    FROM df_dm_dataprep_num_var
    GROUP BY 1
    """)

df_dm_dataprep_num_var_final.createOrReplaceTempView("df_dm_dataprep_num_var_final")





In [ ]:
# 4. Calcula qtd de ciclos e de ids do cliente
df_dm_cycle_var_final = spark.sql("""SELECT ch_cpf_hash
    ,SUM(CASE WHEN id_ciclo = 0 THEN 1 ELSE 0 END) AS qt_id_ciclo_0
    ,SUM(CASE WHEN id_cliente_nivel = 0 THEN 1 ELSE 0 END) AS qt_id_cliente_nivel_0
    ,SUM(CASE WHEN id_ciclo = 1 THEN 1 ELSE 0 END) AS qt_id_ciclo_1
    ,SUM(CASE WHEN id_cliente_nivel = 1 THEN 1 ELSE 0 END) AS qt_id_cliente_nivel_1
    ,SUM(CASE WHEN id_ciclo = 2 THEN 1 ELSE 0 END) AS qt_id_ciclo_2
    ,SUM(CASE WHEN id_cliente_nivel = 2 THEN 1 ELSE 0 END) AS qt_id_cliente_nivel_2
    ,SUM(CASE WHEN id_ciclo = 3 THEN 1 ELSE 0 END) AS qt_id_ciclo_3
    ,SUM(CASE WHEN id_cliente_nivel = 3 THEN 1 ELSE 0 END) AS qt_id_cliente_nivel_3
    ,SUM(CASE WHEN id_cliente_nivel = 4 THEN 1 ELSE 0 END) AS qt_id_cliente_nivel_4
    ,SUM(CASE WHEN id_cliente_nivel = 5 THEN 1 ELSE 0 END) AS qt_id_cliente_nivel_5
    ,SUM(CASE WHEN id_cliente_nivel = 6 THEN 1 ELSE 0 END) AS qt_id_cliente_nivel_6
    ,SUM(CASE WHEN id_cliente_nivel = 7 THEN 1 ELSE 0 END) AS qt_id_cliente_nivel_7
    ,SUM(CASE WHEN id_ciclo IS NULL THEN 1 ELSE 0 END) AS qt_id_ciclo_na
    ,SUM(CASE WHEN id_cliente_nivel IS NULL THEN 1 ELSE 0 END) AS qt_id_cliente_nivel_na
FROM df_dm
GROUP BY 1
""")

df_dm_cycle_var_final.createOrReplaceTempView("df_dm_cycle_var_final")


In [ ]:
# 5. Calcula as flags ao longo do tempo
df_dm_flags_var_final = spark.sql("""
    with
    base_dm_stage as
        (SELECT ch_cpf_hash
            ,dt_fechamento
            ,CAST(SPLIT_PART(dt_fechamento,'-',1) as int) aa_referencia
            ,CAST(SPLIT_PART(dt_fechamento,'-',2) as int) mm_referencia
            ,CASE WHEN ds_behaviour = 'ALTO' THEN 1 ELSE 0 END AS ds_behaviour_ALTO
            ,CASE WHEN ds_behaviour = 'BAIXO' THEN 1 ELSE 0 END AS ds_behaviour_BAIXO
            ,CASE WHEN id_tipo_ultimo_movimento = 'CREDITO' THEN 1 ELSE 0 END AS id_tipo_ultimo_movimento_CREDITO
            ,CASE WHEN id_tipo_ultimo_movimento = 'DEBITO' THEN 1 ELSE 0 END AS id_tipo_ultimo_movimento_DEBITO
            ,CASE WHEN ds_categoria_ultimo_movimento = 'Depósito' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_Deposito
            ,CASE WHEN ds_categoria_ultimo_movimento = 'DMCred' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_DMCred
            ,CASE WHEN ds_categoria_ultimo_movimento = 'Estorno' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_Estorno
            ,CASE WHEN ds_categoria_ultimo_movimento = 'Fraud in Process' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_Fraud_in_Process
            ,CASE WHEN ds_categoria_ultimo_movimento = 'fraud in technology' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_fraud_in_technology
            ,CASE WHEN ds_behaviour = 'MODERADO' THEN 1 ELSE 0 END AS ds_behaviour_MODERADO
            ,CASE WHEN fl_cpf_ativo = 'N' THEN 1 ELSE 0 END AS fl_cpf_ativo_N
            ,CASE WHEN fl_tem_dm_app = 'N' THEN 1 ELSE 0 END AS fl_tem_dm_app_N
            ,CASE WHEN fl_conta_avita = 'N' THEN 1 ELSE 0 END AS fl_conta_avita_N
            ,CASE WHEN fl_tem_dmcred = 'N' THEN 1 ELSE 0 END AS fl_tem_dmcred_N
            ,CASE WHEN fl_tem_dmpag = 'N' THEN 1 ELSE 0 END AS fl_tem_dmpag_N
            ,CASE WHEN fl_dmpag_ativo = 'N' THEN 1 ELSE 0 END AS fl_dmpag_ativo_N
            ,CASE WHEN fl_Farmacia_Ativo = 'N' THEN 1 ELSE 0 END AS fl_Farmacia_Ativo_N
            ,CASE WHEN fl_Mercado_Ativo = 'N' THEN 1 ELSE 0 END AS fl_Mercado_Ativo_N
            ,CASE WHEN fl_Construcao_Ativo = 'N' THEN 1 ELSE 0 END AS fl_Construcao_Ativo_N
            ,CASE WHEN fl_Bandeirado_Ativo = 'N' THEN 1 ELSE 0 END AS fl_Bandeirado_Ativo_N
            ,CASE WHEN fl_Outros_Ativo = 'N' THEN 1 ELSE 0 END AS fl_Outros_Ativo_N
            ,CASE WHEN ds_behaviour IS NULL THEN 1 ELSE 0 END AS ds_behaviour_na
            ,CASE WHEN fl_cpf_ativo IS NULL THEN 1 ELSE 0 END AS fl_cpf_ativo_na
            ,CASE WHEN fl_tem_dm_app IS NULL THEN 1 ELSE 0 END AS fl_tem_dm_app_na
            ,CASE WHEN fl_conta_avita IS NULL THEN 1 ELSE 0 END AS fl_conta_avita_na
            ,CASE WHEN fl_tem_dmcred IS NULL THEN 1 ELSE 0 END AS fl_tem_dmcred_na
            ,CASE WHEN fl_tem_dmpag IS NULL THEN 1 ELSE 0 END AS fl_tem_dmpag_na
            ,CASE WHEN fl_dmpag_ativo IS NULL THEN 1 ELSE 0 END AS fl_dmpag_ativo_na
            ,CASE WHEN fl_Farmacia_Ativo IS NULL THEN 1 ELSE 0 END AS fl_Farmacia_Ativo_na
            ,CASE WHEN fl_Mercado_Ativo IS NULL THEN 1 ELSE 0 END AS fl_Mercado_Ativo_na
            ,CASE WHEN fl_Construcao_Ativo IS NULL THEN 1 ELSE 0 END AS fl_Construcao_Ativo_na
            ,CASE WHEN fl_Bandeirado_Ativo IS NULL THEN 1 ELSE 0 END AS fl_Bandeirado_Ativo_na
            ,CASE WHEN fl_Outros_Ativo IS NULL THEN 1 ELSE 0 END AS fl_Outros_Ativo_na
            ,CASE WHEN id_tipo_ultimo_movimento IS NULL THEN 1 ELSE 0 END AS id_tipo_ultimo_movimento_na
            ,CASE WHEN ds_categoria_ultimo_movimento IS NULL THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_na
            ,CASE WHEN ds_categoria_ultimo_movimento = 'Pagamento' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_Pagamento
            ,CASE WHEN ds_categoria_ultimo_movimento = 'PagamentoFaturaCartao' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_PagamentoFaturaCartao
            ,CASE WHEN ds_categoria_ultimo_movimento = 'personal_loan' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_personal_loan
            ,CASE WHEN ds_categoria_ultimo_movimento = 'PIX' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_PIX
            ,CASE WHEN ds_categoria_ultimo_movimento = 'Recarga' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_Recarga
            ,CASE WHEN fl_cpf_ativo = 'S' THEN 1 ELSE 0 END AS fl_cpf_ativo_S
            ,CASE WHEN fl_tem_dm_app = 'S' THEN 1 ELSE 0 END AS fl_tem_dm_app_S
            ,CASE WHEN fl_conta_avita = 'S' THEN 1 ELSE 0 END AS fl_conta_avita_S
            ,CASE WHEN fl_tem_dmcred = 'S' THEN 1 ELSE 0 END AS fl_tem_dmcred_S
            ,CASE WHEN fl_tem_dmpag = 'S' THEN 1 ELSE 0 END AS fl_tem_dmpag_S
            ,CASE WHEN fl_dmpag_ativo = 'S' THEN 1 ELSE 0 END AS fl_dmpag_ativo_S
            ,CASE WHEN fl_Farmacia_Ativo = 'S' THEN 1 ELSE 0 END AS fl_Farmacia_Ativo_S
            ,CASE WHEN fl_Mercado_Ativo = 'S' THEN 1 ELSE 0 END AS fl_Mercado_Ativo_S
            ,CASE WHEN fl_Construcao_Ativo = 'S' THEN 1 ELSE 0 END AS fl_Construcao_Ativo_S
            ,CASE WHEN fl_Bandeirado_Ativo = 'S' THEN 1 ELSE 0 END AS fl_Bandeirado_Ativo_S
            ,CASE WHEN fl_Outros_Ativo = 'S' THEN 1 ELSE 0 END AS fl_Outros_Ativo_S
            ,CASE WHEN ds_categoria_ultimo_movimento = 'Taxa' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_Taxa
            ,CASE WHEN ds_categoria_ultimo_movimento = 'Transferência' THEN 1 ELSE 0 END AS ds_categoria_ultimo_movimento_Transferencia
            ,ROW_NUMBER() OVER(PARTITION BY ch_cpf_hash ORDER BY dt_fechamento desc) as id_ranking
        FROM df_dm),
    base_recencia as
        (SELECT *
        FROM base_dm_stage
        where id_ranking = 1
        )
    SELECT a.ch_cpf_hash
        ,a.dt_fechamento
        ,a.aa_referencia
        ,a.mm_referencia
        -- Valores no mes de referencia
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_behaviour_ALTO ELSE 0 END) AS ds_behaviour_ALTO_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_behaviour_BAIXO ELSE 0 END) AS ds_behaviour_BAIXO_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.id_tipo_ultimo_movimento_CREDITO ELSE 0 END) AS id_tipo_ultimo_movimento_CREDITO_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.id_tipo_ultimo_movimento_DEBITO ELSE 0 END) AS id_tipo_ultimo_movimento_DEBITO_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_Deposito ELSE 0 END) AS ds_categoria_ultimo_movimento_Deposito_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_DMCred ELSE 0 END) AS ds_categoria_ultimo_movimento_DMCred_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_Estorno ELSE 0 END) AS ds_categoria_ultimo_movimento_Estorno_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_Fraud_in_Process ELSE 0 END) AS ds_categoria_ultimo_movimento_Fraud_in_Process_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_fraud_in_technology ELSE 0 END) AS ds_categoria_ultimo_movimento_fraud_in_technology_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_behaviour_MODERADO ELSE 0 END) AS ds_behaviour_MODERADO_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_cpf_ativo_N ELSE 0 END) AS fl_cpf_ativo_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_tem_dm_app_N ELSE 0 END) AS fl_tem_dm_app_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_conta_avita_N ELSE 0 END) AS fl_conta_avita_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_tem_dmcred_N ELSE 0 END) AS fl_tem_dmcred_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_tem_dmpag_N ELSE 0 END) AS fl_tem_dmpag_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_dmpag_ativo_N ELSE 0 END) AS fl_dmpag_ativo_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Farmacia_Ativo_N ELSE 0 END) AS fl_Farmacia_Ativo_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Mercado_Ativo_N ELSE 0 END) AS fl_Mercado_Ativo_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Construcao_Ativo_N ELSE 0 END) AS fl_Construcao_Ativo_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Bandeirado_Ativo_N ELSE 0 END) AS fl_Bandeirado_Ativo_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Outros_Ativo_N ELSE 0 END) AS fl_Outros_Ativo_N_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_behaviour_na ELSE 0 END) AS ds_behaviour_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_cpf_ativo_na ELSE 0 END) AS fl_cpf_ativo_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_tem_dm_app_na ELSE 0 END) AS fl_tem_dm_app_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_conta_avita_na ELSE 0 END) AS fl_conta_avita_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_tem_dmcred_na ELSE 0 END) AS fl_tem_dmcred_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_tem_dmpag_na ELSE 0 END) AS fl_tem_dmpag_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_dmpag_ativo_na ELSE 0 END) AS fl_dmpag_ativo_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Farmacia_Ativo_na ELSE 0 END) AS fl_Farmacia_Ativo_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Mercado_Ativo_na ELSE 0 END) AS fl_Mercado_Ativo_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Construcao_Ativo_na ELSE 0 END) AS fl_Construcao_Ativo_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Bandeirado_Ativo_na ELSE 0 END) AS fl_Bandeirado_Ativo_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Outros_Ativo_na ELSE 0 END) AS fl_Outros_Ativo_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.id_tipo_ultimo_movimento_na ELSE 0 END) AS id_tipo_ultimo_movimento_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_na ELSE 0 END) AS ds_categoria_ultimo_movimento_na_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_Pagamento ELSE 0 END) AS ds_categoria_ultimo_movimento_Pagamento_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_PagamentoFaturaCartao ELSE 0 END) AS ds_categoria_ultimo_movimento_PagamentoFaturaCartao_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_personal_loan ELSE 0 END) AS ds_categoria_ultimo_movimento_personal_loan_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_PIX ELSE 0 END) AS ds_categoria_ultimo_movimento_PIX_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_Recarga ELSE 0 END) AS ds_categoria_ultimo_movimento_Recarga_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_cpf_ativo_S ELSE 0 END) AS fl_cpf_ativo_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_tem_dm_app_S ELSE 0 END) AS fl_tem_dm_app_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_conta_avita_S ELSE 0 END) AS fl_conta_avita_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_tem_dmcred_S ELSE 0 END) AS fl_tem_dmcred_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_tem_dmpag_S ELSE 0 END) AS fl_tem_dmpag_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_dmpag_ativo_S ELSE 0 END) AS fl_dmpag_ativo_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Farmacia_Ativo_S ELSE 0 END) AS fl_Farmacia_Ativo_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Mercado_Ativo_S ELSE 0 END) AS fl_Mercado_Ativo_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Construcao_Ativo_S ELSE 0 END) AS fl_Construcao_Ativo_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Bandeirado_Ativo_S ELSE 0 END) AS fl_Bandeirado_Ativo_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.fl_Outros_Ativo_S ELSE 0 END) AS fl_Outros_Ativo_S_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_Taxa ELSE 0 END) AS ds_categoria_ultimo_movimento_Taxa_m0
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = 0 THEN b.ds_categoria_ultimo_movimento_Transferencia ELSE 0 END) AS ds_categoria_ultimo_movimento_Transferencia_m0
        -- Ultimos 3 meses
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_behaviour_ALTO ELSE 0 END) AS ds_behaviour_ALTO_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_behaviour_BAIXO ELSE 0 END) AS ds_behaviour_BAIXO_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.id_tipo_ultimo_movimento_CREDITO ELSE 0 END) AS id_tipo_ultimo_movimento_CREDITO_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.id_tipo_ultimo_movimento_DEBITO ELSE 0 END) AS id_tipo_ultimo_movimento_DEBITO_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_Deposito ELSE 0 END) AS ds_categoria_ultimo_movimento_Deposito_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_DMCred ELSE 0 END) AS ds_categoria_ultimo_movimento_DMCred_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_Estorno ELSE 0 END) AS ds_categoria_ultimo_movimento_Estorno_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_Fraud_in_Process ELSE 0 END) AS ds_categoria_ultimo_movimento_Fraud_in_Process_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_fraud_in_technology ELSE 0 END) AS ds_categoria_ultimo_movimento_fraud_in_technology_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_behaviour_MODERADO ELSE 0 END) AS ds_behaviour_MODERADO_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_cpf_ativo_N ELSE 0 END) AS fl_cpf_ativo_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_tem_dm_app_N ELSE 0 END) AS fl_tem_dm_app_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_conta_avita_N ELSE 0 END) AS fl_conta_avita_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_tem_dmcred_N ELSE 0 END) AS fl_tem_dmcred_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_tem_dmpag_N ELSE 0 END) AS fl_tem_dmpag_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_dmpag_ativo_N ELSE 0 END) AS fl_dmpag_ativo_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Farmacia_Ativo_N ELSE 0 END) AS fl_Farmacia_Ativo_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Mercado_Ativo_N ELSE 0 END) AS fl_Mercado_Ativo_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Construcao_Ativo_N ELSE 0 END) AS fl_Construcao_Ativo_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Bandeirado_Ativo_N ELSE 0 END) AS fl_Bandeirado_Ativo_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Outros_Ativo_N ELSE 0 END) AS fl_Outros_Ativo_N_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_behaviour_na ELSE 0 END) AS ds_behaviour_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_cpf_ativo_na ELSE 0 END) AS fl_cpf_ativo_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_tem_dm_app_na ELSE 0 END) AS fl_tem_dm_app_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_conta_avita_na ELSE 0 END) AS fl_conta_avita_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_tem_dmcred_na ELSE 0 END) AS fl_tem_dmcred_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_tem_dmpag_na ELSE 0 END) AS fl_tem_dmpag_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_dmpag_ativo_na ELSE 0 END) AS fl_dmpag_ativo_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Farmacia_Ativo_na ELSE 0 END) AS fl_Farmacia_Ativo_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Mercado_Ativo_na ELSE 0 END) AS fl_Mercado_Ativo_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Construcao_Ativo_na ELSE 0 END) AS fl_Construcao_Ativo_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Bandeirado_Ativo_na ELSE 0 END) AS fl_Bandeirado_Ativo_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Outros_Ativo_na ELSE 0 END) AS fl_Outros_Ativo_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.id_tipo_ultimo_movimento_na ELSE 0 END) AS id_tipo_ultimo_movimento_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_na ELSE 0 END) AS ds_categoria_ultimo_movimento_na_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_Pagamento ELSE 0 END) AS ds_categoria_ultimo_movimento_Pagamento_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_PagamentoFaturaCartao ELSE 0 END) AS ds_categoria_ultimo_movimento_PagamentoFaturaCartao_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_personal_loan ELSE 0 END) AS ds_categoria_ultimo_movimento_personal_loan_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_PIX ELSE 0 END) AS ds_categoria_ultimo_movimento_PIX_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_Recarga ELSE 0 END) AS ds_categoria_ultimo_movimento_Recarga_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_cpf_ativo_S ELSE 0 END) AS fl_cpf_ativo_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_tem_dm_app_S ELSE 0 END) AS fl_tem_dm_app_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_conta_avita_S ELSE 0 END) AS fl_conta_avita_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_tem_dmcred_S ELSE 0 END) AS fl_tem_dmcred_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_tem_dmpag_S ELSE 0 END) AS fl_tem_dmpag_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_dmpag_ativo_S ELSE 0 END) AS fl_dmpag_ativo_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Farmacia_Ativo_S ELSE 0 END) AS fl_Farmacia_Ativo_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Mercado_Ativo_S ELSE 0 END) AS fl_Mercado_Ativo_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Construcao_Ativo_S ELSE 0 END) AS fl_Construcao_Ativo_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Bandeirado_Ativo_S ELSE 0 END) AS fl_Bandeirado_Ativo_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.fl_Outros_Ativo_S ELSE 0 END) AS fl_Outros_Ativo_S_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_Taxa ELSE 0 END) AS ds_categoria_ultimo_movimento_Taxa_m_3
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -3 THEN b.ds_categoria_ultimo_movimento_Transferencia ELSE 0 END) AS ds_categoria_ultimo_movimento_Transferencia_m_3
        -- Ultimos 6 meses
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_behaviour_ALTO ELSE 0 END) AS ds_behaviour_ALTO_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_behaviour_BAIXO ELSE 0 END) AS ds_behaviour_BAIXO_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.id_tipo_ultimo_movimento_CREDITO ELSE 0 END) AS id_tipo_ultimo_movimento_CREDITO_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.id_tipo_ultimo_movimento_DEBITO ELSE 0 END) AS id_tipo_ultimo_movimento_DEBITO_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_Deposito ELSE 0 END) AS ds_categoria_ultimo_movimento_Deposito_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_DMCred ELSE 0 END) AS ds_categoria_ultimo_movimento_DMCred_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_Estorno ELSE 0 END) AS ds_categoria_ultimo_movimento_Estorno_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_Fraud_in_Process ELSE 0 END) AS ds_categoria_ultimo_movimento_Fraud_in_Process_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_fraud_in_technology ELSE 0 END) AS ds_categoria_ultimo_movimento_fraud_in_technology_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_behaviour_MODERADO ELSE 0 END) AS ds_behaviour_MODERADO_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_cpf_ativo_N ELSE 0 END) AS fl_cpf_ativo_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_tem_dm_app_N ELSE 0 END) AS fl_tem_dm_app_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_conta_avita_N ELSE 0 END) AS fl_conta_avita_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_tem_dmcred_N ELSE 0 END) AS fl_tem_dmcred_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_tem_dmpag_N ELSE 0 END) AS fl_tem_dmpag_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_dmpag_ativo_N ELSE 0 END) AS fl_dmpag_ativo_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Farmacia_Ativo_N ELSE 0 END) AS fl_Farmacia_Ativo_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Mercado_Ativo_N ELSE 0 END) AS fl_Mercado_Ativo_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Construcao_Ativo_N ELSE 0 END) AS fl_Construcao_Ativo_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Bandeirado_Ativo_N ELSE 0 END) AS fl_Bandeirado_Ativo_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Outros_Ativo_N ELSE 0 END) AS fl_Outros_Ativo_N_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_behaviour_na ELSE 0 END) AS ds_behaviour_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_cpf_ativo_na ELSE 0 END) AS fl_cpf_ativo_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_tem_dm_app_na ELSE 0 END) AS fl_tem_dm_app_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_conta_avita_na ELSE 0 END) AS fl_conta_avita_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_tem_dmcred_na ELSE 0 END) AS fl_tem_dmcred_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_tem_dmpag_na ELSE 0 END) AS fl_tem_dmpag_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_dmpag_ativo_na ELSE 0 END) AS fl_dmpag_ativo_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Farmacia_Ativo_na ELSE 0 END) AS fl_Farmacia_Ativo_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Mercado_Ativo_na ELSE 0 END) AS fl_Mercado_Ativo_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Construcao_Ativo_na ELSE 0 END) AS fl_Construcao_Ativo_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Bandeirado_Ativo_na ELSE 0 END) AS fl_Bandeirado_Ativo_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Outros_Ativo_na ELSE 0 END) AS fl_Outros_Ativo_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.id_tipo_ultimo_movimento_na ELSE 0 END) AS id_tipo_ultimo_movimento_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_na ELSE 0 END) AS ds_categoria_ultimo_movimento_na_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_Pagamento ELSE 0 END) AS ds_categoria_ultimo_movimento_Pagamento_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_PagamentoFaturaCartao ELSE 0 END) AS ds_categoria_ultimo_movimento_PagamentoFaturaCartao_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_personal_loan ELSE 0 END) AS ds_categoria_ultimo_movimento_personal_loan_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_PIX ELSE 0 END) AS ds_categoria_ultimo_movimento_PIX_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_Recarga ELSE 0 END) AS ds_categoria_ultimo_movimento_Recarga_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_cpf_ativo_S ELSE 0 END) AS fl_cpf_ativo_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_tem_dm_app_S ELSE 0 END) AS fl_tem_dm_app_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_conta_avita_S ELSE 0 END) AS fl_conta_avita_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_tem_dmcred_S ELSE 0 END) AS fl_tem_dmcred_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_tem_dmpag_S ELSE 0 END) AS fl_tem_dmpag_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_dmpag_ativo_S ELSE 0 END) AS fl_dmpag_ativo_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Farmacia_Ativo_S ELSE 0 END) AS fl_Farmacia_Ativo_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Mercado_Ativo_S ELSE 0 END) AS fl_Mercado_Ativo_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Construcao_Ativo_S ELSE 0 END) AS fl_Construcao_Ativo_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Bandeirado_Ativo_S ELSE 0 END) AS fl_Bandeirado_Ativo_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.fl_Outros_Ativo_S ELSE 0 END) AS fl_Outros_Ativo_S_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_Taxa ELSE 0 END) AS ds_categoria_ultimo_movimento_Taxa_m_6
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -6 THEN b.ds_categoria_ultimo_movimento_Transferencia ELSE 0 END) AS ds_categoria_ultimo_movimento_Transferencia_m_6
        -- Ultimos 9 meses
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_behaviour_ALTO ELSE 0 END) AS ds_behaviour_ALTO_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_behaviour_BAIXO ELSE 0 END) AS ds_behaviour_BAIXO_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.id_tipo_ultimo_movimento_CREDITO ELSE 0 END) AS id_tipo_ultimo_movimento_CREDITO_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.id_tipo_ultimo_movimento_DEBITO ELSE 0 END) AS id_tipo_ultimo_movimento_DEBITO_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_Deposito ELSE 0 END) AS ds_categoria_ultimo_movimento_Deposito_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_DMCred ELSE 0 END) AS ds_categoria_ultimo_movimento_DMCred_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_Estorno ELSE 0 END) AS ds_categoria_ultimo_movimento_Estorno_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_Fraud_in_Process ELSE 0 END) AS ds_categoria_ultimo_movimento_Fraud_in_Process_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_fraud_in_technology ELSE 0 END) AS ds_categoria_ultimo_movimento_fraud_in_technology_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_behaviour_MODERADO ELSE 0 END) AS ds_behaviour_MODERADO_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_cpf_ativo_N ELSE 0 END) AS fl_cpf_ativo_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_tem_dm_app_N ELSE 0 END) AS fl_tem_dm_app_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_conta_avita_N ELSE 0 END) AS fl_conta_avita_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_tem_dmcred_N ELSE 0 END) AS fl_tem_dmcred_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_tem_dmpag_N ELSE 0 END) AS fl_tem_dmpag_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_dmpag_ativo_N ELSE 0 END) AS fl_dmpag_ativo_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Farmacia_Ativo_N ELSE 0 END) AS fl_Farmacia_Ativo_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Mercado_Ativo_N ELSE 0 END) AS fl_Mercado_Ativo_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Construcao_Ativo_N ELSE 0 END) AS fl_Construcao_Ativo_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Bandeirado_Ativo_N ELSE 0 END) AS fl_Bandeirado_Ativo_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Outros_Ativo_N ELSE 0 END) AS fl_Outros_Ativo_N_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_behaviour_na ELSE 0 END) AS ds_behaviour_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_cpf_ativo_na ELSE 0 END) AS fl_cpf_ativo_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_tem_dm_app_na ELSE 0 END) AS fl_tem_dm_app_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_conta_avita_na ELSE 0 END) AS fl_conta_avita_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_tem_dmcred_na ELSE 0 END) AS fl_tem_dmcred_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_tem_dmpag_na ELSE 0 END) AS fl_tem_dmpag_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_dmpag_ativo_na ELSE 0 END) AS fl_dmpag_ativo_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Farmacia_Ativo_na ELSE 0 END) AS fl_Farmacia_Ativo_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Mercado_Ativo_na ELSE 0 END) AS fl_Mercado_Ativo_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Construcao_Ativo_na ELSE 0 END) AS fl_Construcao_Ativo_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Bandeirado_Ativo_na ELSE 0 END) AS fl_Bandeirado_Ativo_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Outros_Ativo_na ELSE 0 END) AS fl_Outros_Ativo_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.id_tipo_ultimo_movimento_na ELSE 0 END) AS id_tipo_ultimo_movimento_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_na ELSE 0 END) AS ds_categoria_ultimo_movimento_na_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_Pagamento ELSE 0 END) AS ds_categoria_ultimo_movimento_Pagamento_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_PagamentoFaturaCartao ELSE 0 END) AS ds_categoria_ultimo_movimento_PagamentoFaturaCartao_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_personal_loan ELSE 0 END) AS ds_categoria_ultimo_movimento_personal_loan_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_PIX ELSE 0 END) AS ds_categoria_ultimo_movimento_PIX_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_Recarga ELSE 0 END) AS ds_categoria_ultimo_movimento_Recarga_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_cpf_ativo_S ELSE 0 END) AS fl_cpf_ativo_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_tem_dm_app_S ELSE 0 END) AS fl_tem_dm_app_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_conta_avita_S ELSE 0 END) AS fl_conta_avita_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_tem_dmcred_S ELSE 0 END) AS fl_tem_dmcred_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_tem_dmpag_S ELSE 0 END) AS fl_tem_dmpag_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_dmpag_ativo_S ELSE 0 END) AS fl_dmpag_ativo_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Farmacia_Ativo_S ELSE 0 END) AS fl_Farmacia_Ativo_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Mercado_Ativo_S ELSE 0 END) AS fl_Mercado_Ativo_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Construcao_Ativo_S ELSE 0 END) AS fl_Construcao_Ativo_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Bandeirado_Ativo_S ELSE 0 END) AS fl_Bandeirado_Ativo_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.fl_Outros_Ativo_S ELSE 0 END) AS fl_Outros_Ativo_S_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_Taxa ELSE 0 END) AS ds_categoria_ultimo_movimento_Taxa_m_9
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -9 THEN b.ds_categoria_ultimo_movimento_Transferencia ELSE 0 END) AS ds_categoria_ultimo_movimento_Transferencia_m_9
        -- Ultimos 12 meses
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_behaviour_ALTO ELSE 0 END) AS ds_behaviour_ALTO_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_behaviour_BAIXO ELSE 0 END) AS ds_behaviour_BAIXO_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.id_tipo_ultimo_movimento_CREDITO ELSE 0 END) AS id_tipo_ultimo_movimento_CREDITO_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.id_tipo_ultimo_movimento_DEBITO ELSE 0 END) AS id_tipo_ultimo_movimento_DEBITO_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_Deposito ELSE 0 END) AS ds_categoria_ultimo_movimento_Deposito_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_DMCred ELSE 0 END) AS ds_categoria_ultimo_movimento_DMCred_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_Estorno ELSE 0 END) AS ds_categoria_ultimo_movimento_Estorno_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_Fraud_in_Process ELSE 0 END) AS ds_categoria_ultimo_movimento_Fraud_in_Process_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_fraud_in_technology ELSE 0 END) AS ds_categoria_ultimo_movimento_fraud_in_technology_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_behaviour_MODERADO ELSE 0 END) AS ds_behaviour_MODERADO_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_cpf_ativo_N ELSE 0 END) AS fl_cpf_ativo_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_tem_dm_app_N ELSE 0 END) AS fl_tem_dm_app_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_conta_avita_N ELSE 0 END) AS fl_conta_avita_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_tem_dmcred_N ELSE 0 END) AS fl_tem_dmcred_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_tem_dmpag_N ELSE 0 END) AS fl_tem_dmpag_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_dmpag_ativo_N ELSE 0 END) AS fl_dmpag_ativo_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Farmacia_Ativo_N ELSE 0 END) AS fl_Farmacia_Ativo_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Mercado_Ativo_N ELSE 0 END) AS fl_Mercado_Ativo_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Construcao_Ativo_N ELSE 0 END) AS fl_Construcao_Ativo_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Bandeirado_Ativo_N ELSE 0 END) AS fl_Bandeirado_Ativo_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Outros_Ativo_N ELSE 0 END) AS fl_Outros_Ativo_N_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_behaviour_na ELSE 0 END) AS ds_behaviour_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_cpf_ativo_na ELSE 0 END) AS fl_cpf_ativo_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_tem_dm_app_na ELSE 0 END) AS fl_tem_dm_app_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_conta_avita_na ELSE 0 END) AS fl_conta_avita_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_tem_dmcred_na ELSE 0 END) AS fl_tem_dmcred_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_tem_dmpag_na ELSE 0 END) AS fl_tem_dmpag_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_dmpag_ativo_na ELSE 0 END) AS fl_dmpag_ativo_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Farmacia_Ativo_na ELSE 0 END) AS fl_Farmacia_Ativo_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Mercado_Ativo_na ELSE 0 END) AS fl_Mercado_Ativo_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Construcao_Ativo_na ELSE 0 END) AS fl_Construcao_Ativo_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Bandeirado_Ativo_na ELSE 0 END) AS fl_Bandeirado_Ativo_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Outros_Ativo_na ELSE 0 END) AS fl_Outros_Ativo_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.id_tipo_ultimo_movimento_na ELSE 0 END) AS id_tipo_ultimo_movimento_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_na ELSE 0 END) AS ds_categoria_ultimo_movimento_na_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_Pagamento ELSE 0 END) AS ds_categoria_ultimo_movimento_Pagamento_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_PagamentoFaturaCartao ELSE 0 END) AS ds_categoria_ultimo_movimento_PagamentoFaturaCartao_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_personal_loan ELSE 0 END) AS ds_categoria_ultimo_movimento_personal_loan_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_PIX ELSE 0 END) AS ds_categoria_ultimo_movimento_PIX_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_Recarga ELSE 0 END) AS ds_categoria_ultimo_movimento_Recarga_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_cpf_ativo_S ELSE 0 END) AS fl_cpf_ativo_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_tem_dm_app_S ELSE 0 END) AS fl_tem_dm_app_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_conta_avita_S ELSE 0 END) AS fl_conta_avita_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_tem_dmcred_S ELSE 0 END) AS fl_tem_dmcred_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_tem_dmpag_S ELSE 0 END) AS fl_tem_dmpag_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_dmpag_ativo_S ELSE 0 END) AS fl_dmpag_ativo_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Farmacia_Ativo_S ELSE 0 END) AS fl_Farmacia_Ativo_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Mercado_Ativo_S ELSE 0 END) AS fl_Mercado_Ativo_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Construcao_Ativo_S ELSE 0 END) AS fl_Construcao_Ativo_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Bandeirado_Ativo_S ELSE 0 END) AS fl_Bandeirado_Ativo_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.fl_Outros_Ativo_S ELSE 0 END) AS fl_Outros_Ativo_S_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_Taxa ELSE 0 END) AS ds_categoria_ultimo_movimento_Taxa_m_12
        ,sum(case when (b.aa_referencia *12 + b.mm_referencia) - (a.aa_referencia *12 + a.mm_referencia) = -12 THEN b.ds_categoria_ultimo_movimento_Transferencia ELSE 0 END) AS ds_categoria_ultimo_movimento_Transferencia_m_12
    FROM base_recencia a
    LEFT JOIN base_dm_stage b
    ON a.ch_cpf_hash = b.ch_cpf_hash
    GROUP BY 1,2,3,4
    """)


df_dm_flags_var_final.createOrReplaceTempView("df_dm_flags_var_final")


In [ ]:
# 6. Cria base unificada
df_unificado = df_dm_flags_var_final \
    .join(df_dm_ultimo_num_var_final, on="ch_cpf_hash", how="left") \
    .join(df_dm_primeiro_num_var_final, on="ch_cpf_hash", how="left") \
    .join(df_dm_dataprep_num_var_final, on="ch_cpf_hash", how="left") \
    .join(df_dm_cycle_var_final, on="ch_cpf_hash", how="left")

In [ ]:
df_unificado.show(5, truncate=False)

+----------------------------------------------------------------+-------------+-------------+-------------+--------------------+---------------------+-----------------------------------+----------------------------------+-----------------------------------------+---------------------------------------+----------------------------------------+-------------------------------------------------+----------------------------------------------------+------------------------+-----------------+------------------+-------------------+------------------+-----------------+-------------------+----------------------+---------------------+------------------------+------------------------+--------------------+------------------+------------------+-------------------+--------------------+-------------------+------------------+--------------------+-----------------------+----------------------+-------------------------+-------------------------+---------------------+------------------------------+------

In [ ]:
df_unificado.printSchema()